# Exercise 5: Production Schedule

Problem statement:

## Multi-Product Manufacturing with Setup Costs
A specialty electronics manufacturer produces 4 different circuit board types on
3 production lines over a 5-day planning horizon. They want to minimize total costs (production + setup).

### Production lines:

Line A: High precision, $50/hour, 40 hours available per day
Line B: Standard quality, $35/hour, 48 hours available per day
Line C: High volume, $25/hour, 60 hours available per day

### Products & requirements:

Premium boards: 2 hours/unit, $200 profit each, can only use Line A
Standard boards: 1.5 hours/unit, $120 profit each, can use Line A or B
Basic boards: 1 hour/unit, $80 profit each, can use any line
Prototype boards: 4 hours/unit, $500 profit each, can only use Line A

### Setup costs
There are one-time costs if you produce that product on that line.

* Premium on Line A: $2,000 setup
* Standard on Line A: $1,500 setup, on Line B: $1,000 setup
* Basic on any line: $500 setup
* Prototype on Line A: $3,000 setup

### Customer demands
(minimum quantities needed by end of week):

* Premium: 50 units
* Standard: 80 units
* Basic: 200 units
* Prototype: 15 units

Business rule: Once you start production of a product on a line, you must produce at least 10 units (to justify the setup cost).

Goal: Maximize profit (revenue - production costs - setup costs) while meeting all demands.

## Modelling production lines over time

Let's focus on modelling a single production line, because once we figure that out we can expand that approach to multiple lines.

Modelling the time for a single line, seems there are two major ways we might do this. Producing items takes a variety of times including 1 hour, 2 hours, and 1.5 hours.

* **Periods**. We might model time as half-hour chunks, and within each chunk make *N* boolean variables that says whether the line makes each different item.  We'd make a constraint that says that we are only making a single item at a time.  But we'd also need to make a series of constraints that say that once we start making an item, we have to keep making that item for a certain number of periods until it's done.  This would get challenging to make sure there are no gaps.

* **Production List.** We could model the first time *t* subscript as the *first item* that the line produces, and the second *t* subscript as the *second item* that the line produces.  A given subscript of *t* could represent an hour if the item it's producing takes an hour, and the second subscript might represent 1.5 hours if the second item takes 1.5 hours.  This will make it much easier to represent the constraints if we take this approach.

## Simple demonstration

Let's try a simple demonstration of the idea with 1 production line and 2 products to demonstrate the idea and work out the basics.

For this problem let's say:

1. We have 1 production line
2. We have 2 products:
    * Lamps take 1 hour to make and produce 20 profit.
    * Timers take 1.5 hours to make and produce 40 profit.
3. We must produce at least 2 lamps and 2 timers.
4. We have a single shift of 12 hours.
5. Maximize profit.

This is a super simple example but will help us figure out the modelling strategy.

In [1]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

In [2]:
all_items = ['lamp', 'timer']
profit_by_item = {'lamp': 20, 'timer': 40}
periods_by_item = {'lamp': 2, 'timer': 3} # half-hour periods.
min_demand_by_item = {'lamp': 2, 'timer': 2}
shift_max_periods = 24 # 12 hour total run time max.
max_slots = 12 # A "slot" here means a thing that we made on the line.
all_slots = list(range(max_slots)) # Slot 0 is the first thing we made, slot 1 is the second

In [3]:
# Decision variables
make_vars = {} # Key: (slot, item) - true if we're making this item on the line during this slot.
for slot in all_slots:
    for item in all_items:
        make_vars[slot, item] = solver.BoolVar(f'make_{slot}_{item}')

In [4]:
# Constraint: can only make one item at a time.
for slot in all_slots:
    constraint = solver.Constraint(0, 1, 'slot_{slot}_item') # Up to 1 item can be selected.
    for item in all_items:
        constraint.SetCoefficient(make_vars[slot, item], 1)

In [5]:
# Constraint: minimum production quantities
infinity = solver.infinity()
for item in all_items:
    constraint = solver.Constraint(min_demand_by_item[item], infinity, f'min_demand_{item}')
    for slot in all_slots:
        constraint.SetCoefficient(make_vars[slot, item], 1)

In [6]:
# Constraint: total number of periods used must be <= shift_max_periods (12 hour day)
constraint = solver.Constraint(0, shift_max_periods, f'shift_max_periods')
for item in all_items:
    for slot in all_slots:
        # If we're making this item in this slot, this is how many periods that will take:
        constraint.SetCoefficient(make_vars[slot, item], periods_by_item[item])

In [7]:
# Objective: Maximize profit
objective = solver.Objective()
for slot in all_slots:
    for item in all_items:
        objective.SetCoefficient(make_vars[slot, item], profit_by_item[item])
objective.SetMaximization()

In [8]:
# Solve
result_status = solver.Solve()

In [9]:
is_solved = result_status == pywraplp.Solver.OPTIMAL
if is_solved:
    print('Solved!')
else:
    print('NOT Solved')

Solved!


In [19]:
print(f'Objective value: profit={objective.Value()}')
for slot in all_slots:
    selected_items = [item for item in all_items if make_vars[slot, item].solution_value()]
    if not selected_items:
        continue
    if len(selected_items) > 1:
        print('This should not happen! Have more than one item to be made during this slot')
    item = selected_items[0]
    print(f'{slot+1}. {item}')

Objective value: profit=300.0
1. timer
2. timer
3. timer
4. timer
5. timer
6. timer
7. lamp
8. lamp
9. lamp


So we'll be making 6 timers and 3 lamps. This comes out to exactly the duration of our shift.

Another way of thinking about this data structure:

In [34]:
print('   ' + ' '.join(all_items))
for slot in all_slots:
    item_strs = []
    for item in all_items:
        #print(f'vars[{slot},{item}]={make_vars[slot, item].solution_value()}')
        if make_vars[slot, item].solution_value():
            item_strs.append('[x]')
        else:
            item_strs.append('[ ]')
    item_line = ' '.join(item_strs)
    print(f'{slot + 1:2}. {item_line}')

   lamp timer
 1. [ ] [x]
 2. [ ] [x]
 3. [ ] [x]
 4. [ ] [x]
 5. [ ] [x]
 6. [ ] [x]
 7. [x] [ ]
 8. [x] [ ]
 9. [x] [ ]
10. [ ] [ ]
11. [ ] [ ]
12. [ ] [ ]


## Modelling Production Line Compatibility

I considered a few ways to model the production line compatibility:
1. **Normal constraint approach**.  For products that cannot be produced on a given line, we make constraints that say the sum of those decision variables must be zero.  Consider the item "Premium Boards" which can only be made on line A.  We can introduce constraints that say that the total production of these boards on line B and C must be zero.  
2. **In Production Coefficients**.  When we are going to make an item on a line, we're going to introduce a constraint that says the total items produced >= the demand for that item.  If I line can't manufacture an item, we can multiply it by `can_produce(line, item)` which is 0 or 1, and thus we won't count things in the plan that are produced on an incompatible line. This will make the solver automatically not select these plan items, because they won't satisfy any demand.
3. **"Hard Coded" Decision Variables**.  If a given line can't produce an item, we can put in a numeric 0 instead of a decision variable.  If we use this technique we'll need to remember to check if a variable is a number instead of a decision variable, and either need to use the value as a number directly, or call `.solution_value()` on it.
4. **Sparse Decision Variables**.  The is a variant of solution 3. If we determine that a given line cannot produce a given item, we *don't generate a decision variable for it*.  Then we don't have to make a constraint to eliminate it like we did in solution 1 above.  But we do need to check if `x_vars[line, slot, item] is not None` before we try to use it, since not all of them will be set.

## Modelling One-Time Setup Costs

We have this challenge in the rules of the problem that say that we must pay a one-time setup cost for a given production line if we're going to make any of an item on a production line.

To do this, we'll introduce a set of decision variables, one per combination of line + product, which can be either 0 or 1.  Then we'll add constraints to force those values to be turned on if and only if any of the items are made on those lines.

In simple terms, we want to introduce a set of variables called `pay_setup[line, item]`. Then we want to do something like `IF make[slot, line, item] = 1 THEN pay_setup[line, item] = 1`.  But unfortunately we can't do that directly, so we have to coax the solver into doing that for us by adding constraints.  There's a trick that's commonly used for one-time setup situations like this as follows.

So we'll do 3 steps:
1. **Introduce decision variables** `pay_setup[line, item]` which are 0-1 binary variables indicating if we're paying the setup cost on the given line for the given product.
2. **Introduce constraints.**  We will make a constraint for every `make[slot, line, item] <= pay_setup[line, item]`. This will force the pay_setup to be set to 1 for the line, item if *any* of the `make[slot, line, item]` are 1.  There's actually a trick we must do here, because we must have all the variables on the left side of the inequality. (If we needed variables on both sides of the inequality this would be quadratic instead of linear.) To do that, subtract `pay_setup` from both sides, which is why you'll see the coefficient go negative. So the constraint we'll actually do will be structured as `make[slot, line, item] - pay_setup[line, item] <= 0`. See more details about this constraint expression below.
3. **Add costs to objective function**.  We're actually maximizing profit in this problem, so this means we will need to update our profit objective function to *subtract* the setup costs, by supplying negative coefficients for the setup costs as part of our objective function.

#### The setup cost constraint expression
We said that we will create constraints to ensure that we pay the setup costs.
`make[slot, line, item] - pay_setup[line, item] <= 0`

There are 4 possible values for the `make` and `setup` variables above, since both of those are binary variables:
* `0 - 0 = 0` : Not making anything, not paying setup costs
* `0 - 1 = -1` : Not making anything, but paid setup costs in another slot
* `1 - 0 = 1` : Making something but not paying setup costs. Not OK!
* `1 - 1 = 0` : Making something and paying the setup costs

To express this constraint, we need to set a minimum and maximum value for the constraint.  When we create a constraint we use code like this:
```py
constraint = solver.Constraint(min_value, max_value, name)
```

Note that the min and max values there are *for the expression* not for binary values.  We don't actually care about the minimum value, we can leave that unconstrained.  So we'll pass `-infinity` there to signify to the world that this constraint is not about the lower bound.  We could use -1 there and it would work, but we're using `-infinity` as a signal to the world that we don't care about that half of the constraint.

We will however use `0` as the upper bound, to indicate the this **expression** must come out to at most 0.  This will force the solver to pay the setup costs.


## Important Discovery: Setup Cost Constraints Need Both Directions

After implementing and testing, we discovered that the simple constraint `make[slot, line, item] - pay_setup[line, item] <= 0` is **not sufficient** on its own!

The issue is that this constraint only enforces one direction:
- **IF** we make something, **THEN** we must pay setup

But it doesn't enforce the reverse:
- **IF** we don't make anything, **THEN** we should not pay setup

### Why the optimization doesn't save us

You might think "the solver is maximizing profit, so it won't pay unnecessary setup costs." However, the solver can find multiple optimal solutions, and it might arbitrarily choose one where setup costs are paid even when no production occurs.

### The complete solution

We need **both** constraints to create a proper logical equivalence:

1. **Force setup when producing**: `make[slot, line, part] <= pay_setup[part, line]` for each slot
2. **Prevent setup when not producing**: `pay_setup[part, line] <= sum(make[slot, line, part] for all slots)`

This creates the logical relationship: `pay_setup[part, line] = 1 ⟺ (total production > 0)`

### Implementation

```python
# Constraint 1: If we make anything, we must pay setup
for slot in all_slots:
    constraint = solver.Constraint(-infinity, 0, f'setup_force_{line}_{part}_{slot}')
    constraint.SetCoefficient(x_vars[slot, line, part], 1)
    constraint.SetCoefficient(pay_setup[part, line], -1)

# Constraint 2: If we don't make anything, we don't pay setup  
constraint = solver.Constraint(-infinity, 0, f'setup_prevent_{line}_{part}')
constraint.SetCoefficient(pay_setup[part, line], 1)
for slot in all_slots:
    constraint.SetCoefficient(x_vars[slot, line, part], -1)
```

Without both constraints, you may see spurious setup costs being paid for production lines that aren't actually producing that item!